In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661777 sha256=d61be96af3e60249aded7afc3614e2195507f78bf5867bf0f99405c91f06b8ee
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.2 MB/s eta 0:00:00


In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-4o2k2m5w/kobert-tokenizer_92d5aafbde0f4dd593a6719798987ccf
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-4o2k2m5w/kobert-tokenizer_92d5aafbde0f4dd593a6719798987ccf
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=fcf76858d19fb7e2338396977c5538c1ca8c61c806fa336576aa52ca5409e099
  Stored in directory: /tmp/pip-ephem-wheel-cache-31wy86cy/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
# GPU 사용 시
device = torch.device("cuda:0")

In [7]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from sklearn.model_selection import train_test_split



In [16]:
!cd '/content/drive/MyDrive/final_proj'

In [19]:
df=pd.read_csv('/content/drive/MyDrive/final_proj/multiple_qna_cleaned_4.csv',encoding='UTF-8')

<ipython-input-19-13ec7a93ae7c>:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/drive/MyDrive/final_proj/multiple_qna_cleaned_4.csv',encoding='UTF-8')


In [20]:
df.head(5)

,Unnamed: 0,question,answer,q_mbti_ie,q_mbti_ns,q_mbti_tf,q_mbti_pj,a_mbti_ie,a_mbti_ns,a_mbti_tf,a_mbti_pj
0,0,궁금해요,공방 가서 맞추는 건 싫고 금으로 맞추는 거 아님 싫어요 ㅎ,i,n,f,p,i,n,f,p
1,1,사귄 지 얼마 안 됐고 섬 탈 때도 그냥 용건만 연락하는 편이길래 그러려니 했는데 ...,원하시는 일상 공유가 어떤 건지 궁금해요,i,n,f,p,i,n,f,p
2,2,알려줘야 된다고 생각하세요? 아니면 굳이 필요 없다고 생각하세요? 자잘한 일상 공유...,저는 무조건 ..! 말해야 된다고 생각해요,i,n,f,p,i,n,f,p
3,3,애칭 어떻게 부르셨는지 알려주세요,자기 정도만 쓰는 거 같아요. 애인을 지칭할 때 '너'라고 하기에는 너무 정 ...,i,n,f,p,i,n,f,p
4,4,나만 남자친구랑 데이트 생각하는 거 같고 어찌나 서운하던지 근데 또 집에 가면 편해...,직장 상사 스타일이라고 생각하심 돼요 제가 느끼기에는 f 부분이 너무 적어요....,i,n,f,p,i,n,f,p


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88417 entries, 0 to 88416
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  88417 non-null  int64 
 1   question    31066 non-null  object
 2   answer      31066 non-null  object
 3   q_mbti_ie   61099 non-null  object
 4   q_mbti_ns   61099 non-null  object
 5   q_mbti_tf   61099 non-null  object
 6   q_mbti_pj   61099 non-null  object
 7   a_mbti_ie   61099 non-null  object
 8   a_mbti_ns   61099 non-null  object
 9   a_mbti_tf   61099 non-null  object
 10  a_mbti_pj   61099 non-null  object
dtypes: int64(1), object(10)
memory usage: 7.4+ MB


In [23]:
df.isnull().sum()

Unnamed: 0        0
question      57351
answer        57351
q_mbti_ie     27318
q_mbti_ns     27318
q_mbti_tf     27318
q_mbti_pj     27318
a_mbti_ie     27318
a_mbti_ns     27318
a_mbti_tf     27318
a_mbti_pj     27318
dtype: int64

In [28]:
df_q=df.dropna(subset = ['question','q_mbti_ie'])

In [29]:
df_q.isnull().sum()

Unnamed: 0    0
question      0
answer        0
q_mbti_ie     0
q_mbti_ns     0
q_mbti_tf     0
q_mbti_pj     0
a_mbti_ie     0
a_mbti_ns     0
a_mbti_tf     0
a_mbti_pj     0
dtype: int64

In [30]:
df_q.count()

Unnamed: 0    20661
question      20661
answer        20661
q_mbti_ie     20661
q_mbti_ns     20661
q_mbti_tf     20661
q_mbti_pj     20661
a_mbti_ie     20661
a_mbti_ns     20661
a_mbti_tf     20661
a_mbti_pj     20661
dtype: int64

In [31]:
df_q['q_mbti_ie'].unique()

array(['i', 'e'], dtype=object)

In [32]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df_q['q_mbti_ie'] = encoder.fit_transform(df_q['q_mbti_ie'].values)

<ipython-input-32-540f3193bfd7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q['q_mbti_ie'] = encoder.fit_transform(df_q['q_mbti_ie'].values)


In [33]:
df_q['q_mbti_ie'].unique()

array([1, 0])

In [35]:
# [사용자, 상대방, 상황] data_list 생성
data_list = []
for ques, ans, label in zip (df_q['question'], df_q['answer'], df_q['q_mbti_ie']):
  data = []
  data.append(ques)
  data.append(ans)
  data.append(str(label))

  data_list.append(data)

In [36]:
print(data)
print(data_list[:10])

['때론 책임감이 욕심처럼 보이고 고집스러워 보일 수 있지만 책임감이야말로 우리의 자존심이자.  원동력이 아닐까 생각합니다.  오늘 이렇게 자기 성찰로 하루를 시작합니다 모두 즐거운 하루 보내세요 ^^/ ', '저도 그래요 ㅎㅎ   ', '0']
[['궁금해요  ', '공방 가서 맞추는 건 싫고 금으로 맞추는 거 아님   싫어요 ㅎ   ', '1'], ['사귄 지 얼마 안 됐고 섬 탈 때도 그냥 용건만 연락하는 편이길래 그러려니 했는데 막상 사귀고 일주일 정도 지나니까 맨날 거의 형식적인 연락이 좀 그래서 일상 공유 좀 해줬으면 좋겠는데 그 사람을 굳이 바뀌게 하고 싶진 않은 생각도 들고 그 사람한테 해달라 말을 하는 것도 스트레스인데 어떡하는 게 좋을까요  ', '원하시는 일상 공유가 어떤 건지    궁금해요    ', '1'], ['알려줘야 된다고 생각하세요? 아니면 굳이 필요 없다고 생각하세요? 자잘한 일상 공유 같은 거요 이유도 설명해 주세요 ', '저는 무조건 ..!   말해야 된다고 생각해요    ', '1'], ['애칭 어떻게 부르셨는지 알려주세요 ', "자기 정도만 쓰는 거 같아요.    애인을 지칭할 때 '너'라고 하기에는 너무 정 없어 보여서 쓰기 시작했는데 한두 번이 어려웠지 쓰다 보니 어느 순간 아무렇지 않더라고요    ", '1'], ['나만 남자친구랑 데이트 생각하는 거 같고 어찌나 서운하던지 근데 또 집에 가면 편해하고 너무 좋아하는 게 느껴지니까 편하기 쉴 생각하면 저도 그건 또 좋더라고요   표현방식도 다르고 연애 스타일도 달라서 그런지 남자친구는 해맑게 편하기만 한거 같고 저만 맘 고생하는 거 같고 그래요 ㅎㅎㅎ 남자친구가 예전 같지 않은 마음인지 원래 연애 스타일이 그런 건지 시간이 지나면 서로 맞춰져갈 부분인지 혼자 생각이 많아져서 답답한 마음에 얘기해 봅니다 ㅎㅎ ', '직장 상사 스타일이라고 생각하심 돼요    제가 느끼기에는 f 부분이 너무 적어요..   ', '1'], ['istj와 isfj는 알파벳 한 끗 차이인데 

In [37]:
X_train, X_test = train_test_split(data_list, test_size=0.2, random_state=123)

In [38]:
print(len(X_train), len(X_test))

16528 4133


In [39]:
tok = tokenizer.tokenize

In [43]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
# 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

class BERTDataset(Dataset):
    def __init__(self, dataset, q_idx, a_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab = vocab, pad = pad, pair = pair)

        self.sentences = [transform([i[q_idx], i[a_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))
        return (len(self.labels))

In [44]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [46]:
data_train = BERTDataset(X_train, 0, 1, 2, tok, vocab, max_len, True, False)
data_test = BERTDataset(X_test, 0, 1, 2, tok, vocab, max_len, True, False)

In [47]:
data_train[0]

(array([   2, 3305, 5937, 3672, 2705, 4998, 6383, 3996, 3270, 5655, 6999,
         258, 2146, 1917, 7096, 1023, 7887, 7813,   46, 1698,  517, 5590,
        6855, 7327, 1076, 5760, 3533, 7483, 3149, 4102, 1607, 6314, 7784,
        4223, 6896, 2720,  517, 5960, 7996, 3213, 5859, 3271,   46, 2964,
        1764, 1195, 2266, 2011,  893, 1564, 5439, 1185, 4099,   54,  785,
        7096, 2235, 5793, 7828, 3990, 5760, 5211, 5920,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [48]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [49]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [50]:
model = BERTClassifier(bertmodel, dr_rate = 0.5).to(device)

In [51]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [52]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [53]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [54]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-54-e6a38b13095b>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/259 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 0.7477201223373413 train acc 0.40625
epoch 1 batch id 201 loss 0.6246970295906067 train acc 0.6762282338308457
epoch 1 train acc 0.6815275096525096


<ipython-input-54-e6a38b13095b>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 4 test acc 0.7173466735966736


  0%|          | 0/259 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.5659608244895935 train acc 0.75
epoch 5 batch id 201 loss 0.5996537208557129 train acc 0.7000932835820896
epoch 5 train acc 0.7005912162162162


  0%|          | 0/65 [00:00<?, ?it/s]

epoch 5 test acc 0.7173466735966736


In [55]:
## 학습 모델 저장
PATH = '/content/drive/MyDrive/final_proj' # google 드라이브 연동 해야함. 관련코드는 뺐음
torch.save(model, PATH + 'KoBERT_qna_q_ie.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능